### HW4.1 Build a decision to predict whether you can play tennis or not

In [1]:
!pwd

/Users/z00193k/Desktop/DataScienceTrainingDocs/Assignments


In [2]:
%cd DecisionTree

/Users/z00193k/Desktop/DataScienceTrainingDocs/Assignments/DecisionTree


In [57]:
%%writefile dataset.txt
sunny hot high FALSE no
sunny hot high TRUE no
overcast hot high FALSE yes
rainy mild high FALSE yes
rainy cool normal FALSE yes
rainy cool normal TRUE no
overcast cool normal TRUE yes
sunny mild high FALSE no
sunny cool normal FALSE yes
rainy mild normal FALSE yes
sunny mild normal TRUE yes
overcast mild high TRUE yes
overcast hot normal FALSE yes
rainy mild high TRUE no

Overwriting dataset.txt


In [58]:
f=open("/Users/z00193k/Desktop/DataScienceTrainingDocs/Assignments/DecisionTree/dataset.txt", "r")

In [59]:
splitline=[]
inputs=[]
current={}
for line in f:
    inter=[]
    splitline = line.split()
    current["outlook"]=splitline[0]
    current["temperature"]=splitline[1]
    current["humidity"]=splitline[2]
    current["wind"]=splitline[3]
    inter.append(current)
    inter.append(splitline[4])
#    inter[1]=splitline[5]
    inputs.append(inter)
#    current={}

In [60]:
from __future__ import division
from collections import Counter, defaultdict
from functools import partial
import math, random

def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2) for p in class_probabilities if p)

def class_probabilities(labels):
    total_count = len(labels)
    return [count/total_count for count in Counter(labels).values()]

def data_entropy(labeled_data):        
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

In [61]:
def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)    
    return sum(data_entropy(subset)*len(subset)/total_count for subset in subsets )

In [62]:
inputs = [({'humidity': 'high','outlook': 'rainy','temperature': 'mild','wind': 'TRUE'},'no'),
          ({'humidity': 'high','outlook': 'rainy','temperature': 'mild','wind': 'TRUE'},'no'),
          ({'humidity': 'high','outlook': 'rainy','temperature': 'mild','wind': 'TRUE'},'yes'),
          ({'humidity': 'high','outlook': 'rainy','temperature': 'mild','wind': 'TRUE'},'yes'),
          ({'humidity': 'high','outlook': 'rainy','temperature': 'mild','wind': 'TRUE'},'yes'),
          ({'humidity': 'high','outlook': 'rainy','temperature': 'mild','wind': 'TRUE'},'no'),
          ({'humidity': 'high','outlook': 'rainy','temperature': 'mild','wind': 'TRUE'},'yes'),
          ({'humidity': 'high','outlook': 'rainy','temperature': 'mild','wind': 'TRUE'},'no'),
          ({'humidity': 'high','outlook': 'rainy','temperature': 'mild','wind': 'TRUE'},'yes'),
          ({'humidity': 'high','outlook': 'rainy','temperature': 'mild','wind': 'TRUE'},'yes'),
          ({'humidity': 'high','outlook': 'rainy','temperature': 'mild','wind': 'TRUE'},'yes'),
          ({'humidity': 'high','outlook': 'rainy','temperature': 'mild','wind': 'TRUE'},'yes'),
          ({'humidity': 'high','outlook': 'rainy','temperature': 'mild','wind': 'TRUE'},'yes'),
          ({'humidity': 'high','outlook': 'rainy','temperature': 'mild','wind': 'TRUE'},'no')]

In [63]:
inputs

[({'humidity': 'high',
   'outlook': 'rainy',
   'temperature': 'mild',
   'wind': 'TRUE'},
  'no'),
 ({'humidity': 'high',
   'outlook': 'rainy',
   'temperature': 'mild',
   'wind': 'TRUE'},
  'no'),
 ({'humidity': 'high',
   'outlook': 'rainy',
   'temperature': 'mild',
   'wind': 'TRUE'},
  'yes'),
 ({'humidity': 'high',
   'outlook': 'rainy',
   'temperature': 'mild',
   'wind': 'TRUE'},
  'yes'),
 ({'humidity': 'high',
   'outlook': 'rainy',
   'temperature': 'mild',
   'wind': 'TRUE'},
  'yes'),
 ({'humidity': 'high',
   'outlook': 'rainy',
   'temperature': 'mild',
   'wind': 'TRUE'},
  'no'),
 ({'humidity': 'high',
   'outlook': 'rainy',
   'temperature': 'mild',
   'wind': 'TRUE'},
  'yes'),
 ({'humidity': 'high',
   'outlook': 'rainy',
   'temperature': 'mild',
   'wind': 'TRUE'},
  'no'),
 ({'humidity': 'high',
   'outlook': 'rainy',
   'temperature': 'mild',
   'wind': 'TRUE'},
  'yes'),
 ({'humidity': 'high',
   'outlook': 'rainy',
   'temperature': 'mild',
   'wind': 'TR

In [64]:
def group_by(items, key_fn):
    """returns a defaultdict(list), where each input item 
    is in the list whose key is key_fn(item)"""
    groups = defaultdict(list)
    for item in items:
        key = key_fn(item)
        groups[key].append(item)
    return groups
    
def partition_by(inputs, attribute):
    """returns a dict of inputs partitioned by the attribute
    each input is a pair (attribute_dict, label)"""
    return group_by(inputs, lambda x: x[0][attribute]) 

In [65]:
def partition_entropy_by(inputs, attribute):
    """computes the entropy corresponding to the given partition"""        
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

In [66]:
for key in['humidity','outlook','temperature','wind']:
    print key, partition_entropy_by(inputs, key)

humidity 0.940285958671
outlook 0.940285958671
temperature 0.940285958671
wind 0.940285958671


In [45]:
senior_inputs = [(input, label) for input, label in inputs if input["humidity"] == "high"]

for key in ['outlook', 'temperature', 'wind']:
    print key, partition_entropy_by(senior_inputs, key)

outlook 0.940285958671
temperature 0.940285958671
wind 0.940285958671


In [46]:
def classify(tree, input):
    """classify the input using the given decision tree"""
    
    # if this is a leaf node, return its value
    if tree in [True, False]:
        return tree
   
    # otherwise find the correct subtree
    attribute, subtree_dict = tree
    
    subtree_key = input.get(attribute)  # None if input is missing attribute

    if subtree_key not in subtree_dict: # if no subtree for key,
        subtree_key = None              # we'll use the None subtree
    
    subtree = subtree_dict[subtree_key] # choose the appropriate subtree
    return classify(subtree, input)     # and use it to classify the input

In [47]:
def build_tree_id3(inputs, split_candidates=None):

    # if this is our first pass, 
    # all keys of the first input are split candidates
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()

    # count Trues and Falses in the inputs
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues
    
    if num_trues == 0:                  # if only Falses are left
        return False                    # return a "False" leaf
        
    if num_falses == 0:                 # if only Trues are left
        return True                     # return a "True" leaf

    if not split_candidates:            # if no split candidates left
        return num_trues >= num_falses  # return the majority leaf
                            
    # otherwise, split on the best attribute
    best_attribute = min(split_candidates,
        key=partial(partition_entropy_by, inputs))

    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates 
                      if a != best_attribute]
    
    # recursively build the subtrees
    subtrees = { attribute : build_tree_id3(subset, new_candidates)
                 for attribute, subset in partitions.iteritems() }

    subtrees[None] = num_trues > num_falses # default case

    return (best_attribute, subtrees)

print "building the tree"
tree = build_tree_id3(inputs)
print tree

building the tree
True


In [48]:
print "Junior / Java / tweets / no phd: ", classify(tree, 
     {"humidity" : "high", 
      "outlook" : "rainy", 
      "temperature" : "mild", 
      "wind" : "TRUE"}) 


Junior / Java / tweets / no phd:  True


In [49]:
print "Intern: ", classify(tree, { "level" : "humidity" } )
print "Senior: ", classify(tree, { "level" : "Senior" } )

Intern:  True
Senior:  True
